In [3]:
import gym
import gym_backgammon
from gym_backgammon.envs.backgammon import WHITE, BLACK, COLORS, TOKEN
env = gym.make('gym_backgammon:backgammon-v0')

class TDAgent:
    def __init__(self, color, model):
        self.color = color
        self.model = model
        self.name = 'TDAgent({})'.format(COLORS[color])

    def roll_dice(self):
        return (-randint(1, 6), -randint(1, 6)) if self.color == WHITE else (randint(1, 6), randint(1, 6))

    def choose_best_action(self, actions, env,exploration_rate):
        best_action = None
        observations = []
        if actions:
            actions = list(actions)
            
            # print(exploration_rate)
            if random.uniform(0,1)<exploration_rate:
                best_action = random.choice(actions)
            else:
                tmp_counter = env.counter
                env.counter = 0
                state = env.game.save_state()
                
                # Iterate over all the legal moves and pick the best action
                for i, action in enumerate(actions):
                    observation, reward, done, info = env.step(action)
                    observations.append(observation)
    
                    # restore the board and other variables (undo the action)
                    env.game.restore_state(state)
                observations = np.array(observations)
                print(observations[0])
                
                values = self.model.predict(observations).flatten()
                
                # practical-issues-in-temporal-difference-learning, pag.3
                # ... the network's output P_t is an estimate of White's probability of winning from board position x_t.
                # ... the move which is selected at each time step is the move which maximizes P_t when White is to play and minimizes P_t when Black is to play.
                best_action_index = int(np.argmax(values)) if self.color == WHITE else int(np.argmin(values))
                best_action = actions[best_action_index]
                env.counter = tmp_counter

        return best_action

model = tf.keras.models.load_model("./gym_env_models/backgammon_RLmodel_gym_80_5_10000.h5")
agents = {WHITE: TDAgent(WHITE, model=model), BLACK: TDAgent(BLACK, model=model)}

NameError: name 'model' is not defined